In [1]:
from datetime import datetime
from astra_db import AstraSession
from state import OffsetState as offsetstate

astra = AstraSession()

In [4]:
# Set the current day and time buckets
date = datetime.now()
current_date = int(date.strftime('%Y%m%d')) # bucket by day
current_time = int(date.strftime('%H%M%S%f')[:9]) # format time as HHMMSSsss

# ACTIVITY
cap_type = 'activity'

def get_offset_count(cap_type, activity):
    rows = None
    print()
    state_stmt = "SELECT cap_time, cap_count FROM cap_state WHERE cap_type = %s AND cap_action = %s"
    rows = astra.session.execute(state_stmt, (cap_type, activity))
    if not rows:
        comm_offset_time = 0
        comm_offset_count = 0
        print(f"STATE {cap_type}|{activity} - get offset: No rows returned")
    else:
        comm_offset_time = rows[0].cap_time or 0
        comm_offset_count = rows[0].cap_count or 0
        print(f"STATE {cap_type}|{activity} - get offset: {comm_offset_time}, {comm_offset_count}")
    
    return comm_offset_time, comm_offset_count
    
    
# Iterate through each activity
for activity in ['credit_card', 'mortgage', 'business', 'retail', 'investment']:
    # Total count = offset_count + rows since last offset

    # Get the offset count
    (comm_offset_time, comm_offset_count) = get_offset_count(cap_type, activity)
    """
    rows = None
    print()
    state_stmt = "SELECT cap_time, cap_count FROM cap_state WHERE cap_type = %s AND cap_action = %s"
    rows = astra.session.execute(state_stmt, (cap_type, activity))
    if not rows:
        comm_offset_time = 0
        comm_offset_count = 0
        print(f"STATE {cap_type}|{activity} - get offset: No rows returned")
    else:
        comm_offset_time = rows[0].cap_time or 0
        comm_offset_count = rows[0].cap_count or 0
        print(f"STATE {cap_type}|{activity} - get offset: {comm_offset_time}, {comm_offset_count}")
    """

    # Get the count of rows since last offset
    rows = None
    get_acty_cap_stmt = "SELECT comm_time, comm_offset_time, comm_offset_count FROM actycap_day WHERE activity = %s AND comm_date = %s AND comm_time > %s"
    rows = astra.session.execute(get_acty_cap_stmt, [activity, current_date, comm_offset_time ])
    if not rows:
        row_count = 0
        total_comm_count = comm_offset_count
        print(f"STATE {cap_type}|{activity} - get rows: No rows returned")
    else:
        comm_offset_time = rows[0].comm_offset_time or 0
        #comm_offset_time = rows[0].get("comm_offset_time", 0)
        #comm_offset_time = getattr(rows[0], "comm_offset_time", 0)
        #comm_offset_time = rows[0]["comm_offset_time"] if "comm_offset_time" in rows[0] else 0

        comm_offset_count = rows[0].comm_offset_count or 0
        #comm_offset_count = rows[0].get("comm_offset_count", 0)
        #comm_offset_count = getattr(rows[0], "comm_offset_count", 0)
        #comm_offset_count = rows[0]["comm_offset_count"] if "comm_offset_count" in rows[0] else 0

        comm_time = rows[0].comm_time or 0
        #comm_time = rows[0].get("comm_time", 0)
        #comm_time = getattr(rows[0], "comm_time", 0)
        #comm_time = rows[0]["comm_time"] if "comm_time" in rows[0] else 0

        row_count = sum(1 for _ in rows)  # Count rows

        total_comm_count = int(comm_offset_count) + int(row_count)
        print(f"STATE {cap_type}|{activity} - get rows: {comm_offset_time}, {comm_offset_count}, {comm_time}")

    print(f"STATE {cap_type}|{activity} - Total Count: {total_comm_count},  Row Count: {row_count}")

    # Update State Table
    #state_stmt = "INSERT INTO cap_state (cap_type, cap_action, cap_time, cap_count) VALUES (%s, %s, %s, %s)"
    #astra.session.execute(state_stmt, (cap_type, activity, comm_time, total_comm_count))



STATE activity|credit_card - get offset: No rows returned
STATE activity|credit_card - get rows: No rows returned
STATE activity|credit_card - Total Count: 0,  Row Count: 0

STATE activity|mortgage - get offset: No rows returned
STATE activity|mortgage - get rows: No rows returned
STATE activity|mortgage - Total Count: 0,  Row Count: 0

STATE activity|business - get offset: No rows returned
STATE activity|business - get rows: No rows returned
STATE activity|business - Total Count: 0,  Row Count: 0

STATE activity|retail - get offset: No rows returned
STATE activity|retail - get rows: No rows returned
STATE activity|retail - Total Count: 0,  Row Count: 0

STATE activity|investment - get offset: No rows returned
STATE activity|investment - get rows: No rows returned
STATE activity|investment - Total Count: 0,  Row Count: 0
